### 找工作多工爬蟲

In [95]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import csv
import time
import random
import concurrent.futures
import math
import re
import pandas as pd

In [96]:
def seek_104(url):
    global df
    date = []
    title = []
    company = []
    salary = []
    link = []
    job_desc = []
    location = []
    
    ua = UserAgent()
    headers = {'User-Agent':ua.random}
    
    res = requests.get(url, headers = headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    jobs = soup.find_all('div', class_ = 'b-block__left')
    
    for job in jobs:
        try:
            date.append(job.find('span',class_='b-tit__date').text.replace('\n',''))
        except:
            date.append('-')

        try:  
            title.append(job.find('a',class_='js-job-link').text)
        except:
            title.append('-')

        try:
            company.append(job.find('ul',class_='b-list-inline b-clearfix').text.replace('\n',''))
        except:
            company.append('-')

        try:
            salary.append(job.find('div',class_='job-list-tag b-content').text)
        except:
            salary.append('-')

        try:
            link.append('https:' + job.find('a').get('href'))
        except:
            link.append('-')
        
        try:
            job_desc.append(job.find('p',class_='job-list-item__info b-clearfix b-content').text.replace('\n',' '))
        except:
            job_desc.append('-')
            
        try:
            location.append(job.find('ul',class_='b-list-inline b-clearfix job-list-intro b-content').find("li").text)
        except:
            location.append('-')
    
    data_dict =  {"刊登日期": date, "職稱": title, "公司名稱":company, "公司地點":location,"描述":job_desc, "薪資":salary, "webURL": link} 
    df = df.append(pd.DataFrame(data_dict))
    time.sleep(sleep_time)
    return df

def seek_1111(url):
    global df
    date = []
    title = []
    company = []
    location = []
    salary = []
    link = []
    job_desc = []
    
    ua = UserAgent()
    headers = {'User-Agent':ua.random}
    
    res = requests.get(url, headers = headers)
    soup = BeautifulSoup(res.text, 'html.parser')
    
    jobs = soup.find_all('div', class_ = 'item__job')
    for job in jobs:
        try:
            title.append(job.find('div',class_='item__job-position0 item__m--link').text)
        except:
            title.append('-')

        try:
            link.append('https:' + job.find('a').get('href'))
        except:
            link.append('-')

        try:  
            company.append(job.find('div',class_ = 'item__job-organ-m').get('aria-label'))
        except:
            company.append('-')

        try:
            location.append(job.find('i',class_='item__job-prop-item item__job-prop-workcity').get('aria-label'))
        except:
            location.append('-')

        try:
            salary.append(job.find('i',class_='item__job-prop-item item__job-prop-salary').get('aria-label'))
        except:
            salary.append('-')

        try:
            date.append(job.find('div',class_='item__job-control-item item__job-control-datechange').get('data-mmdd'))
        except:
            date.append('-')
        
        try:
            job_desc.append(job.find('div',class_='item__job-desc item__job-desc-un_extension').get('title').replace('\n',' '))
        except:
            job_desc.append('-')

    data_dict =  {"刊登日期": date, "職稱": title, "公司名稱":company, "公司地點":location,"描述":job_desc, "薪資":salary, "webURL": link} 
    df = df.append(pd.DataFrame(data_dict))
    time.sleep(sleep_time)
    return df


def seek_cakeresume(url):
    global df
    title = []
    link = []
    company = []
    job_desc = []
    date = []
    location = []
    salary = []

    df = pd.DataFrame()

    jobs = soup.find_all('div', class_='job-item')
    jobs2 = soup.find_all('div', class_='meta-container')

    for job in jobs:
        try:
            title.append(job.find('div',class_='job-link-wrapper').text)
        except:
            title.append('-')

        try:
            link.append(job.find('a',class_ = 'job-link').get('href'))
        except:
            link.append('-')

        try:  
            company.append(job.find('h5',class_ = 'page-name').text)
        except:
            company.append('-')

        try:  
            job_desc.append(job.find('p',class_ = 'job-desc').text.replace('\n',''))
        except:
            job_desc.append('-')

        try:
            location.append(job.find('div',class_='location-section').text)
        except:
            location.append('-')

        try:
            salary.append(job.find('span',class_='job-salary').text)
        except:
            salary.append('-')

    for job2 in jobs2:
        try:
            date.append(job2.find('span',class_='update-section').text)
        except:
            date.append('-')

    data_dict =  {"刊登日期": date, "職稱": title, "公司名稱":company, "公司地點":location,"描述":job_desc, "薪資":salary, "webURL": link} 
    df = df.append(pd.DataFrame(data_dict))
    time.sleep(sleep_time)
    return df

In [97]:
def main(web):
    global df, keyword, end_page, urls
    df = pd.DataFrame()
    if web =='104':
        urls = [f'https://www.104.com.tw/jobs/search/?keyword={keyword}&order=1&page={str(i)}&ro=0' for i in range(1,end_page+1)]
    elif web =='cakeresume':
        urls = [f'https://www.cakeresume.com/jobs?q={keyword}&ref=navbar_quick_link_jobs&page={str(i)}' for i in range(1,end_page+1)]
    elif web =='1111':
        urls = [f'https://www.1111.com.tw/search/job?ks={keyword}&fs=1&page={str(i)}' for i in range(1,end_page+1)]

In [88]:
# 執行爬蟲並進行多工處理
def crawler_jobSearch():
    main(web)
    if web =='104':
        with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
            executor.map(seek_104, urls)    
        df.to_csv("104.csv", index=False)
    
    elif web =='cakeresume':
        with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
            executor.map(seek_cakeresume, urls)   
        df.to_csv("cakeresume.csv", index=False)
    
    elif web =='1111':
        with concurrent.futures.ThreadPoolExecutor(max_workers=20) as executor:
            executor.map(seek_1111, urls)  
        df.to_csv("1111.csv", index=False) 

In [92]:
web = '1111' # '104', 'cakeresume', '1111'
keyword = 'python'
end_page = 4
crawler_jobSearch()

In [93]:
df

,刊登日期,職稱,公司名稱,公司地點,描述,薪資,webURL
0,07-05,python工程師,億力資訊股份有限公司,台北市松山區,1.熟悉Git或其他版控工具 (must) 2. 具備2年以上Python程式開發、維護經驗...,面議（經常性薪資4萬/月含以上）,https:https://www.1111.com.tw/job/97374762/
1,07-02,Python工程師,威寶數位科技股份有限公司,基隆市信義區,1. AI演算法之研究、發展、設計、構建、驗證等工作。 2. 對需求模型進行特徵建構、演算法...,月薪 3.5萬~7萬元,https:https://www.1111.com.tw/job/97413223/
2,07-01,Python資深/工程師,春水堂科技娛樂股份有限公司,台北市內湖區,1. 獨立開發Python應用程式與測試除錯 2. 使用Python Web framewo...,面議（經常性薪資4萬/月含以上）,https:https://www.1111.com.tw/job/92257524/
3,07-03,Python全端工程師,宏佳騰動力科技股份有限公司(AEON),新北市板橋區,經歷上百年的機車產業發展，車輛已經具備絕佳的性能與精緻的外型與配備，並正式進入環保節能、安全...,面議（經常性薪資4萬/月含以上）,https:https://www.1111.com.tw/job/91228026/
4,07-03,Python 後端工程師,VIRON,台北市松山區,1. 參與產品需求的分析，根據產品需求制訂技術方案與實現。 2. 負責遊戲服務器端系統設計與...,面議（經常性薪資4萬/月含以上）,https:https://www.1111.com.tw/job/97411840/
...,...,...,...,...,...,...,...
15,07-03,運維工程師,VIRON,台北市內湖區,1、三年到五年互聯網運維維護經驗，要求對亞馬遜，微軟雲，及各種 CDN 能夠 熟練使用。 2...,面議（經常性薪資4萬/月含以上）,https:https://www.1111.com.tw/job/97374716/
16,06-29,Web程式設計師(有經驗者尤佳),國立臺灣大學,台北市大安區,網頁資料庫及應用程式撰寫 擅長工具：Java、Python、PHP、MySQL、HTML、J...,月薪 3.2萬~5.9萬元,https:https://www.1111.com.tw/job/85104872/
17,07-05,Back-end Development Engineer - 台北內湖,創宇管理顧問有限公司,台北市內湖區,"•\tBS in Computer Science, Software Engineerin...",面議（經常性薪資4萬/月含以上）,https:https://www.1111.com.tw/job/92214871/
18,07-04,系統工程師,富邦建設股份有限公司,台北市信義區,1.公司內部資料數據整分析。(運用Power BI視覺化分析與管理建置Dashboard) ...,面議（經常性薪資4萬/月含以上）,https:https://www.1111.com.tw/job/97444921/
